# Despliegue del modelo

In [1]:
#GETTING MY ID:
import json

id = open('my_id.json', 'r')
mi = json.load(id)

**Crear grupo de recursos y cargar modelo**

In [ ]:
# Crear grupo de recursos
from azureml.core import Workspace

ws = Workspace.create(name="wsbank_in",
                      subscription_id = mi["my_id"],
                      resource_group = "rgbank_in",
                      location = "centralindia")

Deploying AppInsights with name wsbankininsightscc5671c9.
Deployed AppInsights with name wsbankininsightscc5671c9. Took 7.36 seconds.
Deploying StorageAccount with name wsbankinstorage759585c75.
Deploying KeyVault with name wsbankinkeyvaultf5a3db9c.
Deployed KeyVault with name wsbankinkeyvaultf5a3db9c. Took 25.92 seconds.
Deployed StorageAccount with name wsbankinstorage759585c75. Took 30.37 seconds.
Deploying Workspace with name wsbank_in.
Deployed Workspace with name wsbank_in. Took 42.82 seconds.


In [ ]:
# Cargar modelo a grupo de recursos
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="bank_model",
                                  workspace=ws)

Registering model bank_model


**Crear archivo score.py**

In [5]:
scorepy = """
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('bank_model')
  model = joblib.load(model_path)

def run(raw_data):
  try:
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    result = model.predict(data)

    return json.dumps({"result": result.tolist()})
  except Exception as e:
    return json.dumps({"error": str(e)})
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

**Crear entorno virtual y dependencias**

In [6]:
from azureml.core.environment import Environment
virtual_env = Environment("env_bank")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pandas',
    'scikit-learn'
    ])

**Configuración de inferencia y despliegue del modelo**

In [7]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name='bankrup-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\Esteban\AppData\Local\Temp\ipykernel_27884\2900293506.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [9]:
# Deployment
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


**Guardar URI para usar el modelo**

In [10]:
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

In [13]:
print(scoring_uri)

http://e4df5a73-6f6c-4ce0-a263-3fdcd7a4479b.centralindia.azurecontainer.io/score
